In [ ]:
from cmless.parse import parse_cmless, parse_dir
from rich import print

In [ ]:
# c = parse_cmless('app/views/special_collections/zoom.md')

In [ ]:
collections = parse_dir('app/views/special_collections')
# collections['zoom']
len(collections)
print(collections)

In [ ]:
from cmless.models import Collection

In [ ]:
c = Collection(**collections['zoom'])

In [ ]:
from os import listdir, path

exhibits = {}

# Parse all the exhibits in the app/views/exhibits directory
for exhibit in listdir('app/views/exhibits'):
    print(exhibit)
    exhibit_path = path.join('app/views/exhibits', exhibit)
    if exhibit.endswith('.md'):        
        exhibits[exhibit] = parse_cmless(exhibit_path)
    if path.isdir(exhibit_path):
        exhibits[exhibit] = {}
        for exhibit_file in listdir(exhibit_path):
            if exhibit_file.endswith('.md'):
                exhibits[exhibit][exhibit_file] = parse_cmless(path.join(exhibit_path, exhibit_file))

In [ ]:
exhibits

In [ ]:
w = parse_cmless('app/views/exhibits/zoom.md')

In [ ]:
len(exhibits)


In [ ]:
import re

def cmless2(file):
    with open(file, 'r') as f:
        md = f.read()
    title_pattern = r"^#\s+(.*)"
    section_pattern = r"^##\s+(.*)"
    titles = re.split(title_pattern, md)
    assert titles[0].strip() == '', "There should be nothing before the first title"
    assert len(titles) == 3, "There should be exactly 3 parts to this match"
    title = titles[1].strip()
    body = titles[2].strip()

    
    sections = re.split(section_pattern, body, flags=re.MULTILINE)
    try:
        # If the first section is a number, it means we have a page number
        page = int(sections[0].strip())
    except ValueError:
        # Nevermind
        page = None
        assert sections[0].strip() == '', "First section should be empty if there's no page number"
            
    # Remove the empty first section
    sections = sections[1:]

    # Turn the remaining sections into a dictionary
    sections = {sections[i].strip(): sections[i + 1].strip() for i in range(0, len(sections), 2)}


    sections = {k: v for k, v in sections.items() if k and v}

    result = {
        'Title': title,
        # 'body': body,
        **sections,
    }
    if page is not None:
        result['Page'] = page
    return result

cmless2('app/views/exhibits/zoom.md')

In [ ]:

with open("app/views/exhibits/conservatism/politics-net.md", "r") as f:
    text = f.read()
sections = re.split(r'(?m)^## ', text)

for i, section in enumerate(sections):
    print(f"Section {i}: {section}\n---")

In [ ]:
re.split(r'(?m)^#\s+(.+$)', text)